# Шаг №1
## Загружаем данные и добавляем все необходимое

Для дальнейшей работы нам нужно конвертировать все даты в datetime формат. Далее, нам нужно будет дать каждому запланированному и осуществленному платежу порядковый номер. Это позволит нам соединить их.
В конце просто несколько quality of life столбцов для анализа.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

orders_df = pd.read_csv('orders.csv')
payments_df = pd.read_csv('payments.csv')
plan_df = pd.read_csv('plan.csv')

# Конвертируем все объекты в datetime для дальнейшей работы
orders_df['created_at'] = pd.to_datetime(orders_df['created_at'])
orders_df['put_at'] = pd.to_datetime(orders_df['put_at'])
orders_df['closed_at'] = pd.to_datetime(orders_df['closed_at'])

payments_df['paid_at'] = pd.to_datetime(payments_df['paid_at'])

plan_df['plan_at'] = pd.to_datetime(plan_df['plan_at'])

# Сорт данных для дальнейшей работы, позволяем мне быстрее проверять, работает ли мой код
payments_df = payments_df.sort_values(by=['order_id', 'paid_at'], ascending=[True, True]).reset_index(drop=True)
plan_df = plan_df.sort_values(by=['order_id', 'plan_at'], ascending=[True, True]).reset_index(drop=True)

# Добавляем нумерацию осуществленных и запланированных платежей. Это позволит нам потом соединить эти данные т.к по дате это не возможно
payments_df['payment_order'] = payments_df['paid_at'].groupby(payments_df['order_id']).rank(method='first')
plan_df['plan_order'] = plan_df['plan_at'].groupby(plan_df['order_id']).rank(method='first')

# Оплата имеет только сумму на этот расчет, а план показывает тотальную запланированную сумму. Добавляю тотальную сумму к оплатам и одиночную к плану. 
payments_df['paid_sum_total'] = payments_df['paid_sum'].groupby(payments_df['order_id']).cumsum()
plan_df['plan_sum'] = plan_df['plan_sum_total'].groupby(plan_df['order_id']).diff().fillna(plan_df['plan_sum_total'])

# Распределим все заказы на 5 категорий по занятой сумме. Это пригодиться в дальнейшем при анализе когорт.
orders_df['loan_cohort'] = pd.qcut(orders_df['issued_sum'], q=5, labels=['Малый займ', 'Средний-малый займ', 'Средний займ', 'Средний-высокий займ', 'Высокий займ'])

# Шаг №2
## Соединяем все данные и добавляем подсчеты

Теперь мы можем соединить все 3 файла через order_id и payment_order/plan_order. Теперь мы можем определить, какие платежи было оплачены вовремя, а какие нет. И на сколько они были задержаны.

In [ ]:
# Пора соединить все доступные данные
total_df = pd.merge(orders_df, payments_df, on='order_id', how='inner')
total_df = pd.merge(total_df, plan_df, left_on=['order_id', 'payment_order'], right_on=['order_id', 'plan_order'], how='left')

# Добавим несколько дополнительных столбцов для дальнейшей визуализации и анализа
total_df['plan_to_paid_diff'] = total_df['plan_sum'] - total_df['paid_sum']
total_df['plan_to_paid_total_diff'] = total_df['plan_sum_total'] - total_df['paid_sum_total']

# Если payment delay больше 0, значит платеж задержался. Если меньше, значит оплата пришла раньше, чем ожидалась. Ну а 0 значит, что все было оплачено по плану
total_df['payment_delay'] = (total_df['paid_at'].dt.floor('d') - total_df['plan_at']).dt.days

# Решил аггрегировать данные по месяцам, но можно выбрать и другие рамки
total_df['plan_month'] = total_df['plan_at'].dt.to_period('M')

total_df['is_overdue'] = total_df['payment_delay'] > 0

display(total_df)

# Шаг №3
## Группируем данные по месяцам

Мы почти готовы к анализу. Я выбрал диапазон в месяц для анализа, но можно анализировать и по дням, неделям, годам...

In [ ]:
monthly_metrics = total_df.groupby('plan_month').agg(
    total_transactions=('order_id', 'count'),
    overdue_transactions=('is_overdue', 'sum'),
    avg_delay=('payment_delay', lambda x: x[x > 0].mean()), # уберем все значения, где оплата была до срока
    avg_advance=('payment_delay', lambda x: x[x < 0].mean()) # обратное от просроченного платежа
).reset_index()

monthly_metrics['avg_advance'] = monthly_metrics['avg_advance'].abs() # Сделал числа положительными, так будет проще читать графики

monthly_metrics['overdue_rate'] = monthly_metrics['overdue_transactions'] / monthly_metrics['total_transactions'] * 100

monthly_metrics = monthly_metrics.iloc[:6] # Решил отрезать данные после Ноября 2022 т.к колличество платежей резко падает ¯\_(ツ)_/¯

display(monthly_metrics)

# График №1
## % просроченных платежей

Данный график показывает, что колличество просроченных платежей уменьшается со временем.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(monthly_metrics['plan_month'].astype(str), monthly_metrics['overdue_rate'], marker='o', linewidth=2)
plt.xlabel('Плановый месяц платежа')
plt.ylabel('% просроченных платежей')
plt.title('Динамика просрочки платежей по месяцам')
plt.ylim(ymin=0)
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# График №2
## Среднее время задержки по платежам

Не только % просроченных платежей падает, но и среднее колличество дней после запланированного платежа.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(monthly_metrics['plan_month'].astype(str), monthly_metrics['avg_delay'], marker='o', color='orange', linewidth=2)
plt.xlabel('Плановый месяц платежа')
plt.ylabel('Среднее количество дней просрочки')
plt.title('Динамика задержки платежей по месяцам')
plt.xticks(rotation=45)
plt.ylim(ymin=0)
plt.grid(True)
plt.tight_layout()
plt.show()

# График №3
## Среднее время досрочной оплаты

Те, кто платят раньше установленного срока, тоже начали платить еще раньше со временем.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(monthly_metrics['plan_month'].astype(str), monthly_metrics['avg_advance'], marker='o', color='orange', linewidth=2)
plt.xlabel('Плановый месяц платежа')
plt.ylabel('Среднее количество дней досрочного платежа')
plt.title('Динамика досрочных платежей по месяцам')
plt.ylim(ymin=0)
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# График №4
## Процент просроченных платежей по сумме займа

В начале мы распределили все наши заказы на 5 категорий, теперь мы можем использовать их для анализа данных.

In [ ]:
# Группируем по когортам и рассчитываем процент просрочек
cohort_metrics = total_df.groupby('loan_cohort').agg(
    total_orders=('order_id', 'count'),
    overdue_payments=('is_overdue', 'sum')
).reset_index()

cohort_metrics['overdue_rate'] = cohort_metrics['overdue_payments'] / cohort_metrics['total_orders'] * 100

В этом графике мы можем увидеть, что чем больше сумма займа, тем меньше шанс, что клиент будет оплачивать поздно. Главной группой риска являются малые и средне-малые займы.

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(cohort_metrics['loan_cohort'], cohort_metrics['overdue_rate'], color='skyblue')
plt.xlabel('Когорта по сумме займа')
plt.ylabel('% просроченных платежей')
plt.title('Процент просрочек по когортам займа')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# График №5
## Процент просроченных платежей по дню иесяца

Давайте выясним, есть ли оптимальный день для запланированного платежа. Может быть есть дни, где люди платят с большей вероятность (после зарплаты, например).

In [ ]:
total_df['day_of_month'] = total_df['plan_at'].dt.strftime('%d')

day_of_month_metrics = total_df.groupby('day_of_month').agg(
    total_transactions=('order_id', 'count'),
    overdue_transactions=('is_overdue', 'sum'),
    avg_delay=('payment_delay', lambda x: x[x > 0].mean()), # уберем все значения, где оплата была до срока
    avg_advance=('payment_delay', lambda x: x[x < 0].mean()) # обратное от просроченного платежа
).reset_index()

day_of_month_metrics['avg_advance'] = day_of_month_metrics['avg_advance'].abs() # Сделал числа положительными, так будет проще читать графики

day_of_month_metrics['overdue_rate'] = day_of_month_metrics['overdue_transactions'] / day_of_month_metrics['total_transactions'] * 100

display(day_of_month_metrics)

Как мы видим, дни в середине месяца показывают себя лучше. Можем предположить, что многие получают зарплату в эти дни, что увеличивает шанс оплаты.

In [ ]:
mean = day_of_month_metrics['overdue_rate'].mean()

plt.figure(figsize=(10, 5))
plt.bar(day_of_month_metrics['day_of_month'], day_of_month_metrics['overdue_rate'], color='skyblue')
plt.xlabel('День месяца')
plt.ylabel('% просроченных платежей')
plt.title('Процент просроченных платежей по дням месяца')
plt.axhline(y=mean, color='red', linestyle='-', label='Среднее значение')
plt.grid(axis='y', linestyle=':')
plt.tight_layout()
plt.legend()
plt.show()